In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import plotly.express as px
from pathlib import Path
from IPython.display import display
from transformers import pipeline
from tqdm.notebook import tqdm
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
pd.options.plotting.backend = "plotly"
%matplotlib inline

In [2]:
data_folder_name = "data/seattle"
csv_files = Path(data_folder_name).glob("*.csv")
csv_files = [x for x in csv_files]

In [4]:
dfs = {}
for x in csv_files:
    dfs[str(x.name).split(".")[0]] = pd.read_csv(x)

In [31]:
df_listings = dfs["listings"].copy()

In [32]:
def preprocess_df_listings(df):
    """
    Preprocess listings dataframe
    Input:
        df: Listings dataframe
    Output:
        df: Cleaned listings dataframe
    """
    
    before_cols = df.shape[1]
    # Remove columns with all missing values
    df = df.dropna(how="all", axis=1)
    # Remove columns with redundant data
    for col in df.columns:
        if len((df[col]).unique())==1:
            df = df.drop(col, axis=1)
    after_cols = df.shape[1]

    # Fix review columns
    review_scores_cols = [x for x in df.columns if "review_scores" in x]
    # Remove all rows with invalid reviews
    df = df.dropna(how="all", subset=review_scores_cols)
    # Remove rows with no number of reviews
    df = df.dropna(how="any", subset=["number_of_reviews"])
    # Fill missing reviews with 0
    df[review_scores_cols] = df[review_scores_cols].fillna(0)
    # Make all review scores out of 10
    df["review_scores_rating"] = df["review_scores_rating"].apply(lambda x: x/10)
    
    # Fix rate columns
    rate_col_list = ["host_response_rate", "host_acceptance_rate"]
    for col in rate_col_list:
        df[col] = df[col].apply(lambda x: float(str(x).replace("%", ""))/100).fillna(0)
    # Fix price columns
    price_col_list = ["price",
                     "weekly_price",
                     "monthly_price",
                     "security_deposit",
                     "cleaning_fee",
                     "extra_people"]
    for col in price_col_list:
        df[col] = df[col].apply(lambda x: float(str(x).replace("$", "").replace(",", ""))).fillna(0)
    # Fix bool columns
    bool_col_list = ["host_is_superhost", 
                     "host_has_profile_pic", 
                     "host_identity_verified", 
                     "require_guest_profile_picture",
                     "require_guest_phone_verification",
                     "instant_bookable"]
    for col in bool_col_list:
        df[col] = df[col].apply(lambda x: x == "t")
    # Get amenities count
    df["num_amenities"] = df["amenities"].apply(lambda x: len(x.split(",")))
    
    return df

In [33]:
df_listings = preprocess_df_listings(df_listings)

In [34]:
def aggregate_review_score(x):
    """
    Aggregate the review score for listings
    Input:
        x: One row from dataframe
    Output:
        out: Aggregated review score
    """
    score = x["review_scores_rating"] + \
            x["review_scores_accuracy"] + \
            x["review_scores_cleanliness"] + \
            x["review_scores_checkin"] + \
            x["review_scores_communication"] + \
            x["review_scores_location"] + \
            x["review_scores_value"]
    score = (score/7)
    
    return score

In [35]:
df_listings["aggregate_score"] = [aggregate_review_score(x) for _, x in df_listings.iterrows()]

In [41]:
amenities_dict = {}
# Get count for each amenity listed
for _, row in df_listings.iterrows():
    amenities = row["amenities"].replace("{", "").replace("}", "").replace("\"", "")
    amenities = amenities.replace("Washer / Dryer", "Washer,Dryer")
    if "," in amenities:
        amenities = amenities.split(",")
    else:
        amenities = [amenities]
    for a in amenities:
        if a != "":
            if a not in amenities_dict:
                amenities_dict[a] = {}
            if row["id"] not in amenities_dict[a]:
                amenities_dict[a][row["id"]] = True

In [42]:
for k in amenities_dict:
    df_listings[k] = df_listings["id"].map(amenities_dict[k]).fillna(False)

In [47]:
df_listings.select_dtypes(include=np.number).columns

Index(['id', 'host_id', 'host_response_rate', 'host_acceptance_rate',
       'host_listings_count', 'host_total_listings_count', 'latitude',
       'longitude', 'accommodates', 'bathrooms', 'bedrooms', 'beds',
       'square_feet', 'price', 'weekly_price', 'monthly_price',
       'security_deposit', 'cleaning_fee', 'guests_included', 'extra_people',
       'minimum_nights', 'maximum_nights', 'availability_30',
       'availability_60', 'availability_90', 'availability_365',
       'number_of_reviews', 'review_scores_rating', 'review_scores_accuracy',
       'review_scores_cleanliness', 'review_scores_checkin',
       'review_scores_communication', 'review_scores_location',
       'review_scores_value', 'calculated_host_listings_count',
       'reviews_per_month', 'num_amenities', 'aggregate_score'],
      dtype='object')

In [48]:
df_listings.select_dtypes(include="object").columns

Index(['listing_url', 'name', 'summary', 'space', 'description',
       'neighborhood_overview', 'notes', 'transit', 'thumbnail_url',
       'medium_url', 'picture_url', 'xl_picture_url', 'host_url', 'host_name',
       'host_since', 'host_location', 'host_about', 'host_response_time',
       'host_thumbnail_url', 'host_picture_url', 'host_neighbourhood',
       'host_verifications', 'street', 'neighbourhood',
       'neighbourhood_cleansed', 'neighbourhood_group_cleansed', 'city',
       'state', 'zipcode', 'smart_location', 'is_location_exact',
       'property_type', 'room_type', 'bed_type', 'amenities',
       'calendar_updated', 'first_review', 'last_review',
       'cancellation_policy'],
      dtype='object')

In [49]:
df_listings.select_dtypes(include="category").columns

Index([], dtype='object')